# New methane lifetime treatment within FaIR

- all species emissions driven
- interactive temperatures
- CMIP6 model calibrations

We might expect historical lifetimes to be marginally off as it's possible temperatures will be too warm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
from fair21.defaults import species_config_from_default

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'CO2_FFI': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'CO2_AFOLU': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'CO2': SpeciesID('CO2', Category.CO2),
    'CH4': SpeciesID('CH4', Category.CH4),
    'N2O': SpeciesID('N2O', Category.N2O),
    'CFC-11': SpeciesID('CFC-11', Category.CFC_11),
    'CFC-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'CFC-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'CFC-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'CFC-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'HCFC-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'HCFC-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'HCFC-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'CCl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'CHCl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'CH2Cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'CH3Cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'CH3CCl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'CH3Br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'Halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'Halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'Halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'CF4': SpeciesID('CF4', Category.F_GAS),
    'C2F6': SpeciesID('C2F6', Category.F_GAS),
    'C3F8': SpeciesID('C3F8', Category.F_GAS),
    'c-C4F8': SpeciesID('C-C4F8', Category.F_GAS),
    'C4F10': SpeciesID('C4F10', Category.F_GAS),
    'C5F12': SpeciesID('C5F12', Category.F_GAS),
    'C6F14': SpeciesID('C6F14', Category.F_GAS),
    'C7F16': SpeciesID('C7F16', Category.F_GAS),
    'C8F18': SpeciesID('C8F18', Category.F_GAS),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'NF3': SpeciesID('NF3', Category.F_GAS),
    'SF6': SpeciesID('SF6', Category.F_GAS),
    'SO2F2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'Sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'BC': SpeciesID('BC', Category.BC),
    'OC': SpeciesID('OC', Category.OC),
    'NH3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'VOC': SpeciesID('VOC', Category.REACTIVE_GAS),
    'CO': SpeciesID('CO', Category.REACTIVE_GAS),
    'NOx': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'NOx_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'lapsi': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'H2O_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
scenarios_to_include = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

In [ ]:
# these will use the dict keys from above rather than the actual species names.
# Remember, order is not important, but we need to have consistency between the order the species are input here and in the
# SpeciesConfigs later on.

# We'll use mixed-case here, because we want to extract these species from the RCMIP database and that's how they're 
# entered there.
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 
    'HFC-365mfc', 'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['CO2', 'ari', 'aci', 'ozone', 'contrails', 'lapsi', 'H2O_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2500'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
        
    # solar and volcanic forcing still a little hacky
    solar_forcing = np.zeros(751)
    solar_forcing[:270] = df_forc['solar'].values
    volcanic_forcing = np.zeros(751)
    volcanic_forcing[:270] = df_forc['volcanic'].values
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
conc_ch4 = {}
for scenario in scenarios_to_include:
    conc_ch4[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models = df['model'].unique()

params = {}

configs = []

seedgen = 1355763
for imodel, model in enumerate(models):
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        config_name = f"{model}_{run}"
        climate_response = ClimateResponse(
            ocean_heat_capacity = df.loc[condition, 'C1':'C3'].values.squeeze(),
            ocean_heat_transfer = df.loc[condition, 'kappa1':'kappa3'].values.squeeze(),
            deep_ocean_efficacy = df.loc[condition, 'epsilon'].values[0],
            gamma_autocorrelation = df.loc[condition, 'gamma'].values[0],
            sigma_eta = df.loc[condition, 'sigma_eta'].values[0],
            sigma_xi = df.loc[condition, 'sigma_xi'].values[0],
            stochastic_run = True,
            seed = seedgen
        )
        seedgen = seedgen+399
        species_config = [species_config_from_default(species) for species in species_to_include]
        
        # edit the methane characteristics
        species_config[2].lifetime = 10.788405534387858
        species_config[2].natural_emissions_adjustment = 19.019783117809567
        species_config[2].soil_lifetime = 185
        
        configs.append(Config(config_name, climate_response, species_config))

In [ ]:
len(species_to_include)

In [ ]:
configs[1].species_configs[2]

In [ ]:
# default
RunConfig()

In [ ]:
# modify
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)
run_config

In [ ]:
start = time.time()
fair = FAIR(scenarios, configs, timestep=1, run_config=run_config)
fair.run()
end = time.time()
print (f"{len(scenarios) * len(configs)} ensemble members in {end - start}s.")

In [ ]:
fair.concentration_array[:,0,0,2,0]

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16,9))
for i in range(8):
    ax[i//4, i%4].plot(fair.concentration_array[:,i,:,2,0]);
    ax[i//4, i%4].plot(conc_ch4[scenarios_to_include[i]][:], color='k')
    ax[i//4, i%4].set_title(scenarios_to_include[i])

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16,9))
for i in range(8):
    ax[i//4, i%4].plot(fair.alpha_lifetime_array[:,i,:,2,0] * fair.lifetime_array[:,0,:,2,0]);
    ax[i//4, i%4].set_title(scenarios_to_include[i])

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.temperature[100:351, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4, i%4].plot(2021, 1.2, 'bx')
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('Temperature anomaly')